# Lab 02: Complexity and the Data Flow

**Course:** Big Data

---

## 👤 Student Information

**Name:** `Your Name Here`

**Date:** `DD/MM/YYYY`

---

**Goal:** Understand computational complexity at scale and optimize data processing patterns.

## Learning Objectives

By the end of this lab, you will be able to:

1. **Experience The Scale Factor**: Feel the difference between N=1,000 and N=1,000,000
2. **Prove Memory Hierarchy Impact**: Demonstrate that RAM is faster than Disk
3. **Apply Big O in Practice**: Transition from O(N²) to O(N log N) or O(N)
4. **Use Profiling Tools**: Identify bottlenecks with `timeit` and `cProfile`

## Instructions

1. **Fill in your information above** before starting the lab
2. Read each cell carefully before running it
3. Implement the **TODO functions** when you see them
4. Run cells **from top to bottom** (Shift+Enter)
5. Check that output makes sense after each cell

---

## 📚 Libraries Used in This Lab

### Core Libraries

- **`pandas`** - Data manipulation (CSV/Parquet reading, chunking)
- **`numpy`** - Random data generation and statistics
- **`time`** - Performance timing with `perf_counter()`
- **`cProfile`** - Python's built-in profiler for identifying bottlenecks
- **`collections`** - High-performance container datatypes (`Counter`)
- **`psutil`** - Memory usage monitoring

### Why These Libraries?

- **cProfile**: Shows you exactly WHERE your code spends time
- **collections.Counter**: O(N) duplicate detection vs O(N²) naive loops
- **psutil**: Real memory measurements to prove RAM vs Disk differences

---

## 💡 The Red Zone: Working at Scale

In this lab, we work with **1,000,000 rows** — the "Red Zone" where:

- O(N²) algorithms become **painfully slow** (1 trillion operations!)
- O(N) algorithms remain **fast** (only 1 million operations)
- Memory management becomes **critical**

This is where Big Data thinking starts!

---

## 1. Imports and Setup

In [ ]:
import json
import time
import cProfile
import pstats
import io
from pathlib import Path
from collections import Counter

import pandas as pd
import numpy as np
import psutil

print("✓ All imports successful!")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

## 2. Define Paths

In [ ]:
# Base directories
DATA_RAW = Path("../data/raw")
RESULTS_DIR = Path("../results")

# File paths for this lab
USER_LOGS_PATH = DATA_RAW / "user_logs_1m.csv"
METRICS_PATH = RESULTS_DIR / "lab02_metrics.json"

# Ensure directories exist
DATA_RAW.mkdir(parents=True, exist_ok=True)
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

print("Paths defined:")
print(f"  User Logs: {USER_LOGS_PATH}")
print(f"  Metrics: {METRICS_PATH}")

---

## 3. Dataset Generation: Enter the Red Zone 🔴

First, we need to generate a **1 million row** synthetic dataset.

This dataset simulates user activity logs with:
- `user_id`: Random user IDs (1 to 50,000) — some users appear multiple times!
- `session_id`: Unique session identifier
- `action`: Random action type ("click", "view", "purchase", "scroll", "search")
- `timestamp`: Sequential timestamps
- `value`: Random numeric value (e.g., time spent, amount)

### TODO 1: `generate_user_logs()`

Generate a synthetic dataset with 1 million rows.

**💡 Hints:**
- Use `np.random.seed(seed)` for reproducibility
- Use `np.random.randint(1, 50001, size=n_rows)` for user_ids (creates duplicates!)
- Use `np.arange(n_rows)` for session_ids (unique)
- Use `np.random.choice([...], size=n_rows)` for actions
- Use `pd.date_range()` for timestamps
- Save with `df.to_csv(path, index=False)`

In [ ]:
def generate_user_logs(path: Path, n_rows: int = 1_000_000, seed: int = 42) -> dict:
    """
    Generate a synthetic user logs dataset.
    
    Args:
        path: Where to save the CSV
        n_rows: Number of rows (default 1 million)
        seed: Random seed for reproducibility
    
    Returns:
        Dictionary with: {"rows": int, "cols": int, "size_mb": float}
    """
    # TODO: Implement this function
    # Step 1: Set random seed
    # Step 2: Generate user_id (1 to 50000, allows duplicates)
    # Step 3: Generate session_id (0 to n_rows-1, unique)
    # Step 4: Generate action (random choice from list)
    # Step 5: Generate timestamp (date_range)
    # Step 6: Generate value (random float 0-1000)
    # Step 7: Create DataFrame
    # Step 8: Save to CSV
    # Step 9: Return metadata
    pass

In [ ]:
# TEST: Generate the dataset (only if it doesn't exist)
if not USER_LOGS_PATH.exists():
    print("🔴 Entering the Red Zone: Generating 1 million rows...")
    start = time.perf_counter()
    metadata = generate_user_logs(USER_LOGS_PATH)
    elapsed = time.perf_counter() - start
    print(f"Generated in {elapsed:.2f} seconds")
    print(f"Dataset: {metadata}")
    assert metadata["rows"] == 1_000_000, "Should have 1M rows"
    print("✓ generate_user_logs() works correctly!")
else:
    size_mb = USER_LOGS_PATH.stat().st_size / 1_000_000
    print(f"Dataset already exists: {size_mb:.2f} MB")

---

## Exercise 1: Search Efficiency — O(N) vs O(1) 🔍

**The Question**: How fast can we check if a value exists in a collection?

We'll compare:
- **List**: `x in my_list` → O(N) — must scan every element
- **Set**: `x in my_set` → O(1) — hash lookup, instant!

At N=1,000,000, this difference is **dramatic**.

### TODO 2: `benchmark_search()`

Compare search performance in a List vs a Set.

**💡 Hints:**
- Create both a list and set with the same data: `list(range(n))`
- Search for random keys using `np.random.randint()`
- Use `time.perf_counter()` to measure each search
- Return the median time for each structure

In [ ]:
def benchmark_search(n: int = 1_000_000, n_searches: int = 1000, seed: int = 42) -> dict:
    """
    Benchmark search performance: List vs Set.
    
    Args:
        n: Size of the collection
        n_searches: Number of searches to perform
        seed: Random seed
    
    Returns:
        Dictionary with:
            - "list_median_ms": median search time in List (milliseconds)
            - "set_median_ms": median search time in Set (milliseconds)
            - "speedup": how many times faster Set is
    """
    # TODO: Implement this function
    # Step 1: Create list and set with range(n)
    # Step 2: Generate random keys to search for
    # Step 3: Time each search in the list, collect times
    # Step 4: Time each search in the set, collect times
    # Step 5: Calculate medians and speedup
    pass

In [ ]:
# TEST: Benchmark search
print("Benchmarking List vs Set search (1M elements, 1000 searches)...")
print("This may take a moment...\n")

search_results = benchmark_search(n=1_000_000, n_searches=1000)

print(f"List median search time: {search_results['list_median_ms']:.4f} ms")
print(f"Set median search time:  {search_results['set_median_ms']:.6f} ms")
print(f"\n🚀 Set is {search_results['speedup']:.0f}x faster!")

assert search_results["speedup"] > 100, "Set should be at least 100x faster"
print("\n✓ benchmark_search() works correctly!")

### 💡 Key Insight: Hash Tables

Sets and Dictionaries use **hash tables** for O(1) lookup:

| Structure | Lookup | Memory | Use Case |
|-----------|--------|--------|----------|
| List | O(N) | Low | Ordered data, iteration |
| Set | O(1) | Higher | Membership testing |
| Dict | O(1) | Highest | Key-value mapping |

**Rule of thumb**: If you're checking `x in collection` repeatedly, use a Set!

---

## Exercise 2: The Data Flow — Memory Hierarchy 📊

**The Memory Pyramid**:
```
       ┌─────────┐
       │  L1/L2  │  ← Fastest (nanoseconds)
       ├─────────┤
       │   RAM   │  ← Fast (microseconds)
       ├─────────┤
       │   SSD   │  ← Slower (milliseconds)
       ├─────────┤
       │   HDD   │  ← Slowest (milliseconds)
       └─────────┘
```

We'll prove this by comparing three ways to process our 1M row CSV:

1. **Full Load**: Load entire file into RAM
2. **Chunking**: Load in chunks (controlled memory)
3. **Line Iterator**: Read line by line (minimal memory)

### TODO 3: `load_full()`, `load_chunked()`, `load_iterator()`

Implement three different loading strategies.

**💡 Hints:**
- `pd.read_csv(path)` for full load
- `pd.read_csv(path, chunksize=N)` for chunking (returns iterator)
- `open(path)` and `for line in f:` for line iterator
- Use `psutil.Process().memory_info().rss` to measure memory

In [ ]:
def get_memory_mb() -> float:
    """Get current process memory usage in MB."""
    return psutil.Process().memory_info().rss / 1_000_000


def load_full(path: Path) -> dict:
    """
    Load entire CSV into memory.
    
    Returns:
        {"rows": int, "time_sec": float, "memory_mb": float}
    """
    # TODO: Implement
    # 1. Record start memory and time
    # 2. Load with pd.read_csv()
    # 3. Record end memory and time
    # 4. Return metrics
    pass


def load_chunked(path: Path, chunksize: int = 50_000) -> dict:
    """
    Load CSV in chunks and process.
    
    Returns:
        {"rows": int, "time_sec": float, "peak_memory_mb": float}
    """
    # TODO: Implement
    # 1. Record start time
    # 2. Iterate through chunks: for chunk in pd.read_csv(path, chunksize=chunksize)
    # 3. Count rows, track peak memory
    # 4. Return metrics
    pass


def load_iterator(path: Path) -> dict:
    """
    Load CSV line by line (minimal memory).
    
    Returns:
        {"rows": int, "time_sec": float, "memory_mb": float}
    """
    # TODO: Implement
    # 1. Record start time
    # 2. Open file and count lines (skip header)
    # 3. Record end time and memory
    # 4. Return metrics
    pass

In [ ]:
# TEST: Compare loading strategies
print("Comparing data loading strategies...\n")

print("1. Full Load (all into RAM)...")
full_result = load_full(USER_LOGS_PATH)
print(f"   Rows: {full_result['rows']:,}")
print(f"   Time: {full_result['time_sec']:.3f} sec")
print(f"   Memory: {full_result['memory_mb']:.1f} MB\n")

print("2. Chunked Load (50k at a time)...")
chunked_result = load_chunked(USER_LOGS_PATH)
print(f"   Rows: {chunked_result['rows']:,}")
print(f"   Time: {chunked_result['time_sec']:.3f} sec")
print(f"   Peak Memory: {chunked_result['peak_memory_mb']:.1f} MB\n")

print("3. Line Iterator (minimal memory)...")
iterator_result = load_iterator(USER_LOGS_PATH)
print(f"   Rows: {iterator_result['rows']:,}")
print(f"   Time: {iterator_result['time_sec']:.3f} sec")
print(f"   Memory: {iterator_result['memory_mb']:.1f} MB\n")

print("✓ Data flow comparison complete!")

### 💡 Key Insight: Trade-offs

| Method | Speed | Memory | Use Case |
|--------|-------|--------|----------|
| Full Load | Fastest | Highest | Data fits in RAM, need random access |
| Chunking | Medium | Controlled | Large files, aggregate operations |
| Iterator | Slowest | Minimal | Huge files, streaming processing |

**Rule**: Always know your data size relative to available RAM!

---

## Exercise 3: Identifying the Bottleneck 🔥

**The Problem**: You have a slow function. Where is the time going?

We'll use **profiling** to find the "hot" lines of code.

Here's a deliberately **bad** function that finds duplicate user_ids:

In [ ]:
def find_duplicates_slow(data: list) -> list:
    """
    Find duplicate values in a list.
    
    WARNING: This is O(N²) — VERY SLOW for large data!
    
    Args:
        data: List of values to check
    
    Returns:
        List of duplicate values
    """
    duplicates = []
    for i in range(len(data)):
        for j in range(i + 1, len(data)):
            if data[i] == data[j] and data[i] not in duplicates:
                duplicates.append(data[i])
    return duplicates


print("Slow function defined. Let's profile it!")

### TODO 4: Profile the slow function

Use `cProfile` to identify where the function spends time.

**💡 Hints:**
- Use `cProfile.Profile()` to create a profiler
- Use `profiler.enable()` and `profiler.disable()` to wrap the function call
- Use `pstats.Stats(profiler)` to analyze results
- Sort by `'cumulative'` time to see the biggest bottlenecks

In [ ]:
def profile_function(fn, *args, **kwargs) -> tuple:
    """
    Profile a function and return timing info.
    
    Args:
        fn: Function to profile
        *args, **kwargs: Arguments to pass to the function
    
    Returns:
        Tuple of (result, stats_string, total_time)
    """
    # TODO: Implement profiling
    # 1. Create profiler: pr = cProfile.Profile()
    # 2. Enable profiler: pr.enable()
    # 3. Call the function: result = fn(*args, **kwargs)
    # 4. Disable profiler: pr.disable()
    # 5. Create stats: stats = pstats.Stats(pr)
    # 6. Sort by cumulative time: stats.sort_stats('cumulative')
    # 7. Capture stats to string using io.StringIO()
    # 8. Return result, stats string, and total time
    pass

In [ ]:
# TEST: Profile the slow function with a SMALL sample
# Using 10,000 items (not 1M!) because O(N²) = 100 million operations
print("Loading sample data for profiling...")
sample_df = pd.read_csv(USER_LOGS_PATH, nrows=10_000)
sample_users = sample_df['user_id'].tolist()
print(f"Sample size: {len(sample_users):,} users\n")

print("Profiling find_duplicates_slow()...")
print("(This will take 10-30 seconds due to O(N²) complexity)\n")

result, stats_string, total_time = profile_function(find_duplicates_slow, sample_users)

print(f"Found {len(result)} duplicate user_ids")
print(f"Total time: {total_time:.2f} seconds\n")
print("Profile output (top 10 lines):")
print(stats_string)

print("\n✓ Profiling complete!")

### 💡 Reading the Profile Output

The profile shows:
- **ncalls**: Number of times the function was called
- **tottime**: Time spent IN the function (excluding sub-calls)
- **cumtime**: Total time INCLUDING sub-calls

**The bottleneck**: The nested loops (`for i... for j...`) create O(N²) comparisons!

For N=10,000: N² = 100,000,000 comparisons
For N=1,000,000: N² = 1,000,000,000,000 comparisons (IMPOSSIBLE!)

---

## Exercise 4: The 10x Challenge 🏆

**Your Mission**: Refactor `find_duplicates_slow()` to be at least **10x faster**.

**Strategies**:
1. **Hashing**: Use `collections.Counter` for O(N) counting
2. **Sorting**: Sort first, then scan for adjacent duplicates O(N log N)

You must achieve at least **10x speedup** to pass this exercise.
Aim for **100x or more**!

### TODO 5: `find_duplicates_fast()`

Implement a fast duplicate finder using hashing.

**💡 Hints:**
- Use `collections.Counter(data)` to count occurrences in O(N)
- Filter for items with count > 1
- Return as a list

In [ ]:
def find_duplicates_fast(data: list) -> list:
    """
    Find duplicate values in a list using hashing.
    
    This is O(N) — much faster than the nested loop approach!
    
    Args:
        data: List of values to check
    
    Returns:
        List of duplicate values
    """
    # TODO: Implement using Counter
    # 1. Count occurrences: counts = Counter(data)
    # 2. Filter for duplicates: [item for item, count in counts.items() if count > 1]
    # 3. Return the list
    pass

In [ ]:
# TEST: Compare slow vs fast
print("Comparing slow vs fast duplicate finding...\n")

# Time the slow version (small sample)
print("Slow version (10,000 items):")
start = time.perf_counter()
slow_result = find_duplicates_slow(sample_users)
slow_time = time.perf_counter() - start
print(f"  Time: {slow_time:.3f} seconds")
print(f"  Found: {len(slow_result)} duplicates\n")

# Time the fast version (same sample)
print("Fast version (10,000 items):")
start = time.perf_counter()
fast_result = find_duplicates_fast(sample_users)
fast_time = time.perf_counter() - start
print(f"  Time: {fast_time:.6f} seconds")
print(f"  Found: {len(fast_result)} duplicates\n")

# Calculate speedup
speedup = slow_time / fast_time
print(f"🚀 Speedup: {speedup:.0f}x faster!")

# Verify results match
assert set(slow_result) == set(fast_result), "Results should match!"
assert speedup >= 10, f"Need at least 10x speedup, got {speedup:.1f}x"

print("\n✓ The 10x Challenge: PASSED!")

### Bonus: Scale to 1 Million!

Now let's see the fast version handle the **full dataset**:

In [ ]:
# Load full dataset
print("Loading full dataset (1M rows)...")
full_df = pd.read_csv(USER_LOGS_PATH)
all_users = full_df['user_id'].tolist()
print(f"Loaded {len(all_users):,} user_ids\n")

# Time the fast version on full data
print("Running fast duplicate finder on 1M items...")
start = time.perf_counter()
duplicates = find_duplicates_fast(all_users)
elapsed = time.perf_counter() - start

print(f"Time: {elapsed:.3f} seconds")
print(f"Found: {len(duplicates):,} duplicate user_ids")
print(f"\n🎉 Processed 1 MILLION items in under {elapsed:.1f} seconds!")

# How long would the slow version take?
estimated_slow = (slow_time / 10_000**2) * 1_000_000**2
print(f"\n⚠️  Estimated time for slow version: {estimated_slow/3600:.1f} HOURS!")

---

## 7. Reflection

**Your task:** Write a short reflection (3-5 sentences) answering:

1. What was the most surprising performance difference you observed?
2. How will this change how you write code in the future?
3. When would you choose chunking over full loading?

In [ ]:
# TODO: Write your reflection here
reflection = """
Replace this text with your reflection.
Think about what you learned about complexity at scale.
What will you remember about O(N) vs O(N²)?
When will you use Sets instead of Lists?
""".strip()

print("Your reflection:")
print(reflection)

---

## 8. Save Results

Finally, save all metrics to `results/lab02_metrics.json`.

In [ ]:
# Compile all results
results = {
    "lab": "02_complexity_dataflow",
    "timestamp": pd.Timestamp.now().isoformat(),
    "dataset": {
        "rows": 1_000_000,
        "path": str(USER_LOGS_PATH),
    },
    "exercise_1_search": {
        "list_median_ms": search_results["list_median_ms"],
        "set_median_ms": search_results["set_median_ms"],
        "speedup": search_results["speedup"],
    },
    "exercise_2_dataflow": {
        "full_load_sec": full_result["time_sec"],
        "full_load_memory_mb": full_result["memory_mb"],
        "chunked_sec": chunked_result["time_sec"],
        "chunked_peak_memory_mb": chunked_result["peak_memory_mb"],
        "iterator_sec": iterator_result["time_sec"],
    },
    "exercise_4_optimization": {
        "slow_time_sec": slow_time,
        "fast_time_sec": fast_time,
        "speedup": speedup,
    },
    "reflection": reflection,
}

# Save to JSON
with open(METRICS_PATH, "w") as f:
    json.dump(results, f, indent=2)

print(f"✓ Results saved to: {METRICS_PATH}")

---

## 🎉 Lab Complete!

### What You Learned

1. **Scale matters**: O(N²) is fine for 1,000 items, catastrophic for 1,000,000
2. **Use the right data structure**: Sets give O(1) lookup vs Lists O(N)
3. **Memory hierarchy is real**: Full load vs chunking vs streaming
4. **Profile before optimizing**: Find the actual bottleneck first!
5. **Hashing is magic**: Counter turns O(N²) into O(N)

### Files to Submit

1. `notebooks/lab02_complexity_dataflow.ipynb` (this notebook, with all cells executed)
2. `results/lab02_metrics.json` (generated by this notebook)

---

**Next Lab**: We'll explore parallel processing and distributed computing!